<a href="https://colab.research.google.com/github/keelinarseneault/ML-Engineering/blob/main/Capstone_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import os

path = 'drive/MyDrive/Images'
train_file = 'drive/MyDrive/train.csv'
train = pd.read_csv(train_file)
test_file = 'drive/MyDrive/test.csv'
test = pd.read_csv(test_file)

In [3]:
test.head()

,id
0,test_data_v2/1a2d9fd3e21b4266aea1f66b30aed157.jpg
1,test_data_v2/ab5df8f441fe4fbf9dc9c6baae699dc7.jpg
2,test_data_v2/eb364dd2dfe34feda0e52466b7ce7956.jpg
3,test_data_v2/f76c2580e9644d85a741a42c6f6b39c0.jpg
4,test_data_v2/a16495c578b7494683805484ca27cf9f.jpg


In [4]:
train.head()

,Unnamed: 0,file_name,label
0,0,train_data/a6dcb93f596a43249135678dfcfc17ea.jpg,1
1,1,train_data/041be3153810433ab146bc97d5af505c.jpg,0
2,2,train_data/615df26ce9494e5db2f70e57ce7a3a4f.jpg,1
3,3,train_data/8542fe161d9147be8e835e50c0de39cd.jpg,0
4,4,train_data/5d81fa12bc3b4cea8c94a6700a477cf2.jpg,1


In [5]:
print(f'Training dataset shape: {train.shape}')

train = train[['file_name', 'label']]
train.columns = ['id', 'label']

Training dataset shape: (79950, 3)


In [6]:
test.columns

Index(['id'], dtype='object')

In [7]:
def image_exists(id):
    filepath = f"drive/MyDrive/Images/{id}"
    return os.path.isfile(filepath)

In [8]:
train = train[train["id"].apply(image_exists)]
test = test[test["id"].apply(image_exists)]

In [10]:
print(train.shape)
print(test.shape)

(11040, 2)
(0, 1)
